In [1]:
train_dir = r"C:\Users\rafci\Desktop\tensorflow-speech-recognition-challenge\train"

In [2]:
import numpy as np
import os

In [3]:
X_train = np.load("data/X_train.npy")
y_train = np.load("data/y_train.npy")

X_val = np.load("data/X_val.npy")
y_val = np.load("data/y_val.npy")

X_train = X_train.reshape((-1, X_train.shape[1], X_train.shape[2]))
X_val = X_val.reshape((-1, X_val.shape[1], X_val.shape[2]))

classes = os.listdir(train_dir+'/audio/')
NB_CLASSES = len(classes)
classes

def convert_list_dict(lst):
    res_dct = {i: val for i, val in enumerate(lst)}
    return res_dct
         
classes_index = convert_list_dict(classes)
classes_index

{0: 'bed',
 1: 'bird',
 2: 'cat',
 3: 'dog',
 4: 'down',
 5: 'eight',
 6: 'five',
 7: 'four',
 8: 'go',
 9: 'happy',
 10: 'house',
 11: 'left',
 12: 'marvin',
 13: 'nine',
 14: 'no',
 15: 'off',
 16: 'on',
 17: 'one',
 18: 'right',
 19: 'seven',
 20: 'sheila',
 21: 'silence',
 22: 'six',
 23: 'stop',
 24: 'three',
 25: 'tree',
 26: 'two',
 27: 'up',
 28: 'wow',
 29: 'yes',
 30: 'zero'}

In [4]:
import torch.nn.functional as F

import torch

y_train = torch.tensor(y_train).long()
y_val = torch.tensor(y_val).long()
X_train = torch.tensor(X_train)
X_val = torch.tensor(X_val)



In [7]:
import torch

# Assuming y_train is a PyTorch tensor
y_train_silence = torch.zeros(y_train.shape[0],2).long()
# y_train_silence = torch.where(y_train == 21, torch.tensor([[0, 1]]).repeat(y_train.shape[0], 1), torch.tensor([[1, 0]]).repeat(y_train.shape[0], 1))
y_train_silence[y_train == 21] = torch.tensor([[0, 1]])
y_train_silence[y_train != 21] = torch.tensor([[1, 0]])

X_train_audio = torch.tensor(X_train[y_train != 21])
y_train_audio = torch.tensor(y_train[y_train != 21])

y_train_audio[y_train_audio>21] -= 1

y_val_silence = torch.tensor(y_val)

y_val_silence[y_val==21] = 1
y_val_silence[y_val!=21] = 0


X_val_audio = torch.tensor(X_val[y_val != 21])
y_val_audio = torch.tensor(y_val[y_val != 21])

y_train_audio = torch.nn.functional.one_hot(y_train_audio, num_classes=NB_CLASSES-1)


# print(y_train_silence[:10])
# print(y_train_audio[:10])

C:\Users\rafci\AppData\Local\Temp\ipykernel_11836\2738941801.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_audio = torch.tensor(X_train[y_train != 21])
C:\Users\rafci\AppData\Local\Temp\ipykernel_11836\2738941801.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_audio = torch.tensor(y_train[y_train != 21])
C:\Users\rafci\AppData\Local\Temp\ipykernel_11836\2738941801.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val_silence = torch.tensor(y_val)
C:\Users\rafci\AppData\Local\Temp\ipykernel_11836\2738941801.

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
import numpy as np
import time

class InceptionModule(nn.Module):
    def __init__(self, in_channels, nb_filters=32, bottleneck_size=32*4, kernel_sizes=[41]):
        super(InceptionModule, self).__init__()
        self.bottleneck_size = bottleneck_size
        
        if bottleneck_size and in_channels > 1:
            self.bottleneck = nn.Conv1d(in_channels, bottleneck_size, kernel_size=1)
        
        self.conv_layers = nn.ModuleList()
        for kernel_size in kernel_sizes:
            self.conv_layers.append(nn.Conv1d(bottleneck_size if bottleneck_size else in_channels,
                                              nb_filters, kernel_size=kernel_size, padding='same'))
        
        self.max_pool = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        self.conv_6 = nn.Conv1d(in_channels, nb_filters, kernel_size=1)
        self.batch_norm = nn.BatchNorm1d(nb_filters * len(kernel_sizes) + nb_filters)
        
        # Initialize weights
        self._initialize_weights()
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
    def forward(self, x):
        input_inception = self.bottleneck(x) if hasattr(self, 'bottleneck') else x
        conv_outputs = [conv_layer(input_inception) for conv_layer in self.conv_layers]
        x = self.max_pool(x)
        x = self.conv_6(x)
        conv_outputs.append(x)
        #[print(x.shape) for x in conv_outputs]
        output = torch.cat(conv_outputs, dim=1)
        output = self.batch_norm(output)
        output = F.relu(output)
        return output

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, nb_filters):
        super(ResidualBlock, self).__init__()
        self.conv = nn.Conv1d(in_channels, nb_filters, kernel_size=1)
        self.batch_norm = nn.BatchNorm1d(nb_filters)
        
    def forward(self, x):
        shortcut = self.conv(x)
        shortcut = self.batch_norm(shortcut)
        x = x + shortcut
        x = F.relu(x)
        return x

class ClassifierInception(nn.Module):
    def __init__(self, input_shape, nb_classes, nb_filters=32, use_residual=True, use_bottleneck=True,
                 depth=10, kernel_size=41, bottleneck_size=32):
        super(ClassifierInception, self).__init__()
        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size
        self.bottleneck_size = bottleneck_size
        
        self.conv1 = nn.Conv1d(input_shape[0], nb_filters * 4, kernel_size=kernel_size, padding=kernel_size//2, padding_mode='reflect')
        
        inception_modules = [InceptionModule(nb_filters * 4, nb_filters, bottleneck_size, [kernel_size // (2 ** i) for i in range(3)]) 
                             for _ in range(depth)]
        self.inception_modules = nn.Sequential(*inception_modules)
        
        if use_residual:
            self.residual_blocks = nn.ModuleList([ResidualBlock(nb_filters * 4, nb_filters * 4) for _ in range(depth // 3)])
        
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(nb_filters * 4, nb_classes)
        
        # Initialize weights
        self._initialize_weights()
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.inception_modules(x)
        
        if self.use_residual:
            for i, block in enumerate(self.residual_blocks):
                if (i + 1) % 3 == 0:
                    x = block(x)
        
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [9]:
INPUT_SHAPE = (85, 122)
print(X_train.shape)
BATCH_SIZE = 64
NB_EPOCHS=10

torch.Size([58284, 122, 85])


In [17]:
# Model, criterion, optimizer
model_silence = ClassifierInception(INPUT_SHAPE, 2).to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer_silence = optim.Adam(model_silence.parameters(),lr=0.0001)


# Data loaders
train_silence_dataset = TensorDataset(X_train.float().transpose(2, 1).to('cuda'), y_train_silence.float().to('cuda'))
train_silence_loader = DataLoader(train_silence_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset_silence = TensorDataset(X_val.float().transpose(2, 1).to('cuda'), y_val_silence.float().to('cuda'))
val_loader_silence = DataLoader(val_dataset_silence, batch_size=BATCH_SIZE, shuffle=False)

NB_EPOCHS=3
# Training loop
for epoch in range(NB_EPOCHS):
    model_silence.train()
    for inputs, labels in train_silence_loader:
        optimizer_silence.zero_grad()
        outputs = model_silence(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_silence.step()
    
    
    model_silence.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for inputs, labels in val_loader_silence:
            outputs = model_silence(inputs)
            _, predicted = torch.max(outputs, 1)
            # _, labels = torch.max(labels, 1)            
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
        
        accuracy = total_correct / total_samples
        print(f'Epoch [{epoch+1}/{NB_EPOCHS}], Validation Accuracy: {accuracy:.4f}')

print("Training finished.")

Epoch [1/3], Validation Accuracy: 0.9977
Epoch [2/3], Validation Accuracy: 0.9940
Epoch [3/3], Validation Accuracy: 0.9942
Training finished.


In [23]:
# Model, criterion, optimizer
model_audio = ClassifierInception(INPUT_SHAPE, NB_CLASSES-1).to('cuda')
optimizer_audio = optim.Adam(model_audio.parameters(),lr=0.0005)

train_audio_dataset = TensorDataset(X_train_audio.float().transpose(2, 1).to('cuda'), y_train_audio.float().to('cuda'))
train_audio_loader = DataLoader(train_audio_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset_audio = TensorDataset(X_val_audio.float().transpose(2, 1).to('cuda'), y_val_audio.float().to('cuda'))
val_loader_audio = DataLoader(val_dataset_audio, batch_size=BATCH_SIZE, shuffle=False)


NB_EPOCHS=10
# Training loop
for epoch in range(NB_EPOCHS):
    model_audio.train()
    for inputs, labels in train_audio_loader:
        optimizer_audio.zero_grad()
        outputs = model_audio(inputs)
        #print(outputs.shape,labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_audio.step()
    
    
    model_audio.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for inputs, labels in val_loader_audio:
            outputs = model_audio(inputs)
            _, predicted = torch.max(outputs, 1)
            # _, labels = torch.max(labels, 1)
            predicted[predicted>=21]+=1
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
        
        accuracy = total_correct / total_samples
        print(f'Epoch [{epoch+1}/{NB_EPOCHS}], Validation Accuracy: {accuracy:.4f}')

print("Training finished.")

Epoch [1/10], Validation Accuracy: 0.8452
Epoch [2/10], Validation Accuracy: 0.8854
Epoch [3/10], Validation Accuracy: 0.9120
Epoch [4/10], Validation Accuracy: 0.9134
Epoch [5/10], Validation Accuracy: 0.9195
Epoch [6/10], Validation Accuracy: 0.9282
Epoch [7/10], Validation Accuracy: 0.9223
Epoch [8/10], Validation Accuracy: 0.9365
Epoch [9/10], Validation Accuracy: 0.9281
Epoch [10/10], Validation Accuracy: 0.9279
Training finished.


In [24]:
# Model, criterion, optimizer
model_audio = ClassifierInception(INPUT_SHAPE, NB_CLASSES-1).to('cuda')
optimizer_audio = optim.Adam(model_audio.parameters(),lr=0.0001)

train_audio_dataset = TensorDataset(X_train_audio.float().transpose(2, 1).to('cuda'), y_train_audio.float().to('cuda'))
train_audio_loader = DataLoader(train_audio_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset_audio = TensorDataset(X_val_audio.float().transpose(2, 1).to('cuda'), y_val_audio.float().to('cuda'))
val_loader_audio = DataLoader(val_dataset_audio, batch_size=BATCH_SIZE, shuffle=False)


NB_EPOCHS=10
# Training loop
for epoch in range(NB_EPOCHS):
    model_audio.train()
    for inputs, labels in train_audio_loader:
        optimizer_audio.zero_grad()
        outputs = model_audio(inputs)
        #print(outputs.shape,labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_audio.step()
    
    
    model_audio.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for inputs, labels in val_loader_audio:
            outputs = model_audio(inputs)
            _, predicted = torch.max(outputs, 1)
            # _, labels = torch.max(labels, 1)
            predicted[predicted>=21]+=1
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
        
        accuracy = total_correct / total_samples
        print(f'Epoch [{epoch+1}/{NB_EPOCHS}], Validation Accuracy: {accuracy:.4f}')

print("Training finished.")

Epoch [1/10], Validation Accuracy: 0.8420
Epoch [2/10], Validation Accuracy: 0.8875
Epoch [3/10], Validation Accuracy: 0.9026
Epoch [4/10], Validation Accuracy: 0.9129
Epoch [5/10], Validation Accuracy: 0.9172
Epoch [6/10], Validation Accuracy: 0.9123
Epoch [7/10], Validation Accuracy: 0.9270
Epoch [8/10], Validation Accuracy: 0.9159
Epoch [9/10], Validation Accuracy: 0.9262
Epoch [10/10], Validation Accuracy: 0.9309
Training finished.


In [25]:
val_dataset = TensorDataset(X_val.float().transpose(2, 1).to('cuda'), y_val.float().to('cuda'))
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

model_audio.eval()
model_silence.eval()
with torch.no_grad():
    total_correct = 0
    total_samples = 0
    for inputs, labels in val_loader:
        outputs_silence = model_silence(inputs)
        _, predicted_silence = torch.max(outputs_silence, 1)
        
        outputs_audio = model_audio(inputs[predicted_silence==0])
        _, predicted_audio = torch.max(outputs_audio, 1)
        predicted_audio[predicted_audio>=21]+=1
        
        predicted = torch.tensor(predicted_silence)
        
        predicted[predicted_silence==0] = predicted_audio
        predicted[predicted_silence==1] = 21
        
        
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)
    
    accuracy = total_correct / total_samples
    print(f'Epoch [{epoch+1}/{NB_EPOCHS}], Validation Accuracy: {accuracy:.4f}')

C:\Users\rafci\AppData\Local\Temp\ipykernel_11836\2182270843.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted = torch.tensor(predicted_silence)


Epoch [10/10], Validation Accuracy: 0.9254


c:\Users\rafci\anaconda3\Lib\site-packages\torch\nn\modules\conv.py:303: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(F.pad(input, self._reversed_padding_repeated_twice, mode=self.padding_mode),
